In [4]:
import os
import pickle
import numpy as np
from collections import defaultdict
import re
import torch
import networkx as nx
from hyperbolicity.utils import create_log_dir, setup_logger, str2bool, soft_max, datasp, sample_batch_indices, batched_datasp_submatrices, construct_weighted_matrix, make_batches

In [2]:
import numpy as np
from scipy.spatial import distance as ssd
from scipy.cluster.hierarchy import linkage

def linkage_to_distance_matrix(Z):
    """
    Converts a linkage matrix Z (from scipy.cluster.hierarchy.linkage)
    into a full pairwise distance matrix D of shape (n, n), where n is the number of leaves.
    """
    N = Z.shape[0] + 1
    clusters = [[i] for i in range(N)]
    D = np.zeros((N, N))
    for i in range(N - 1):
        j, k = int(Z[i, 0]), int(Z[i, 1])
        for x in clusters[j]:
            for y in clusters[k]:
                D[x, y] = D[y, x] = Z[i, 2]
        clusters.append(clusters[j] + clusters[k])
    return D

def gromov_tree(distance_matrix, root):
    """
    Computes a Gromov-style distance matrix from a given root using a hierarchical clustering
    approximation based on the induced ultrametric.

    Parameters:
        distance_matrix (np.ndarray): Original pairwise distance matrix.
        root (int): Index to use as the root node.

    Returns:
        np.ndarray: Gromov-adjusted distance matrix (tree-metric approximation).
    """
    n = distance_matrix.shape[0]
    d_root = distance_matrix[root]
    d_max = d_root.max()
    
    gp = np.tile(d_root, (n, 1)) + np.tile(d_root.reshape(n, 1), (1, n)) - distance_matrix
    gp /= 2.0

    d_U = d_max - gp
    np.fill_diagonal(d_U, 0)

    Z = linkage(ssd.squareform(d_U), method='single')
    D_gromov = linkage_to_distance_matrix(Z)

    gp_T = d_max - D_gromov
    d_T = np.tile(d_root, (n, 1)) + np.tile(d_root.reshape(n, 1), (1, n)) - 2.0 * gp_T
    np.fill_diagonal(d_T, 0)

    return d_T

In [3]:
base_path = '/share/home/houedry/projects/hyperbolic/DifferentiableHyperbolicity/hyperbolicity/datasets'

c_elegan = 'D_celegan.pkl'
c_elegan_path = os.path.join(base_path, c_elegan)
with open(c_elegan_path, 'rb') as f:
    distances = pickle.load(f)

In [ ]:
def compute_metrics(distances, weights):
    num_nodes = distances.shape[0]
    edges = torch.triu_indices(num_nodes, num_nodes, offset=1)
    optimum = construct_weighted_matrix(weights, num_nodes, edges)
    G = nx.from_numpy_array(optimum.numpy())
    intermediate_distances = nx.floyd_warshall_numpy(G)
    mean_optim = []
    mean_no_optim = []
    for j in range(num_nodes):
        tree_optim = gromov_tree(intermediate_distances, j)
        mean_optim.append(np.abs(tree_optim-distances).max())
        

    return np.mean(mean_optim), np.min(mean_optim)

In [34]:
base_path = '/share/home/houedry/projects/hyperbolic/DifferentiableHyperbolicity/hyperbolicity/results_expes/expe_test/expe_celegan/expe_celegan/'

def extract_config_key(folder_name):
    return re.sub(r'-run_number-\d+', '', folder_name)

results_dict = defaultdict(list)

print("== Starting experiment scan ==")
print(f"Looking in: {base_path}")

for folder in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder)
    if os.path.isdir(folder_path) and folder.startswith('launch_distance_hyperbolicity_learning'):
        config_key = extract_config_key(folder)

        for file in os.listdir(folder_path):
            if file == "res.pickle":
                with open(os.path.join(folder_path, file), 'rb') as f:
                    weights = pickle.load(f)['weights']
                    try:
                        print(f"    → Computing metrics...")
                        m1= compute_metrics(distances, weights)
                        print(f"    → m1 = {m1}")
                        results_dict[config_key].append(m1)
                    except Exception as e:
                        print(f"[WARN] Failed to process {file} in {folder}: {e}")

if not results_dict:
    print("No results collected. Please double-check path, file names, and formats.")
else:
    print("Results collected! Printing aggregated metrics:\n")

for config, values in results_dict.items():
    values = np.array(values)
    mean_metrics = np.mean(values, axis=0)
    std_metrics = np.std(values, axis=0)
    
    print(f"{config}")
    print(f"\tMetric 1 -> Mean: {mean_metrics[0]:.4f}, Std: {std_metrics[0]:.4f}")
    print(f"\tMetric 2 -> Mean: {mean_metrics[1]:.4f}, Std: {std_metrics[1]:.4f}")


== Starting experiment scan ==
Looking in: /share/home/houedry/projects/hyperbolic/DifferentiableHyperbolicity/hyperbolicity/results_expes/expe_test/expe_celegan/expe_celegan/
    → Computing metrics...
[WARN] Failed to process res.pickle in launch_distance_hyperbolicity_learning_2025_04_15_18_02_04-run_number-4-dataset-celegan-learning_rate-0.5-distance_reg-0.0001-scale_sp-10000.0-scale_delta-10000.0-scale_softmax-10000.0-epochs-500-batch_size-32: 'float' object has no attribute 'device'
    → Computing metrics...
    → m1 = 3.327450295201445
    → Computing metrics...
    → m1 = 3.3273908426012615
    → Computing metrics...
    → m1 = 3.327221460574496
    → Computing metrics...
[WARN] Failed to process res.pickle in launch_distance_hyperbolicity_learning_2025_04_15_19_21_41-run_number-2-dataset-celegan-learning_rate-1.0-distance_reg-0.0001-scale_sp-10000.0-scale_delta-10000.0-scale_softmax-10000.0-epochs-500-batch_size-32: 'float' object has no attribute 'device'
    → Computing met

KeyboardInterrupt: 